# Sơ bộ về URIs (Uniform Resource Identifiers)

URI là địa chỉ tham chiếu đến nơi lưu trữ dữ liệu của bạn. Để Azure Machine Learning có thể kết nối đến dữ liệu của bạn, bạn cần thêm tiền tố URI vào giao thức thích hợp. Có 3 giao thức bạn có thể làm việc với data ở trong AzureML:
- ```http(s)```: Sử dụng cho dữ liệu được lưu (publicly or privately) trong Azure Blob Storage hoặc địa chỉ http(s) publicly khả dụng
- ```abfs(s)```: Sử dụng cho dữ liệu được lưu ở trong Azure Data Lake Storage Gen 2
- ```azureml```: Sử dụng cho dữ liệu được lưu trong datastore

> **Chú ý**: Hãy chắc chắn rằng bạn đã cài đặt AzureML SDK python. Chạy lệnh dưới để xác nhận

In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.14.0
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: d:\anaconda\envs\azureml\Lib\site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, opencensus-ext-azure, opencensus-ext-logging, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

subscription_id = os.getenv("SUBSCRIPTION_ID", default="<my-subscription-id>")
resource_group = os.getenv("RESOURCE_GROUP", default="<my-resource-group>")
workspace_name = os.getenv("WORKSPACE_NAME", default="<my-workspace-name>")
workspace_region = os.getenv("WORKSPACE_REGION", default="eastus2")
storage_account_name = os.getenv("STORAGE_ACCOUNT_NAME", default="<my-storage-account-name>")
container_name = os.getenv("CONTAINER_NAME", default="<my-container-name>")
storage_account_key = os.getenv("STORAGE_ACCOUNT_KEY", default="<my-storage-account-key>")

---
# Kết nối với Azure Machine Learning Client
- credential: Là gói xác thực hiện tại đang để Default

Hãy chắc chắn rằng bạn đã xác thực cả AzureCLI ```azd auth login```

In [3]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

ml_client = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

# Kiểm tra danh sách datastores đã được khởi tạo.

Khi bạn tạo AzureML workspace thì một Azure Storage account cũng được khởi tạo theo. Storage Account bao gồm blob và file storage và được tự động kết nối với workspace của bạn dưới dạng kho dữ liệu.

In [4]:
stores = ml_client.datastores.list()
for ds_name in stores:
    print(ds_name.name)

blob_example
azureml_globaldatasets
workspaceworkingdirectory
workspaceartifactstore
workspaceblobstore
workspacefilestore


---
# Khởi tạo datastore

Mỗi khi bạn muốn kết nối với một dịch vụ Azure Storage nào khác với AzureML workspace bạn cần phải tạo datastore. chú ý rằng khi bạn tạo datastore sẽ tạo kết nối giữa workspace của bạn và bộ lưu trữ chứ ko tạo ra dịch vụ lưu trữ.

Để tạo một datastore và kết nối đến nó bạn cần chỉ định các thông tin:
- name: tên hiển thị của datastore trong AzureML workspace
- description: Mô tả về khi dữ liệu
- account_name: tên của Azure Storage Account
- container_name:: tên của container đến blob store trong AzureStorageAccount
- credentials: Auth

In [5]:
from azure.ai.ml.entities import AzureBlobDatastore
from azure.ai.ml.entities import AccountKeyConfiguration

blob_datastore = AzureBlobDatastore(
    name="blob_example",
    description="Datastore pointing to a blob container",
    account_name=storage_account_name,
    container_name=container_name,
    credentials=AccountKeyConfiguration(account_key=storage_account_key),
)
ml_client.create_or_update(blob_datastore)

AzureBlobDatastore({'type': <DatastoreType.AZURE_BLOB: 'AzureBlob'>, 'name': 'blob_example', 'description': 'Datastore pointing to a blob container', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/da6080d2-3021-495c-8ecc-44feb23b1080/resourceGroups/Machine_Learning_dev01/providers/Microsoft.MachineLearningServices/workspaces/Machine_Experiment/datastores/blob_example', 'Resource__source_path': '', 'base_path': 'g:\\AzureML_CouresVN\\03', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x000002A25A3A2F10>, 'credentials': {'type': 'account_key'}, 'container_name': 'data-container', 'account_name': 'machineestorageadc326696', 'endpoint': 'core.windows.net', 'protocol': 'https'})

In [6]:
stores = ml_client.datastores.list()
for ds_name in stores:
    print(ds_name.name)

blob_example
azureml_globaldatasets
workspaceworkingdirectory
workspaceartifactstore
workspaceblobstore
workspacefilestore


# Khởi tạo Data assets

Là một data scientes bạn muốn tập trung vào việc huấn luyện mô hình học máy. Vì điều đó nên bạn cần truy cập vào data để làm đầu vào cho mô hình học máy của bạn nhưng bạn không hề muốn lo lắng về việc làm sao để truy cập vào dữ liệu đó lúc này chính là lúc bạn cần đến data assets. Bạn có thể dùng data assets để truy câph vào data trong datastores, Azure Storage Account, punlic URLs hoặc là data được lưu trữ trên máy làm việc của bạn.

Lợi ích của việc sử dụng data assets:

- Bạn có thể chia sẻ và tái sử dụng dữ liệu với các thành viên khác trong team của bạn mà họ không cần phải nhớ vị trí của dữ liệu
- Bạn có thể truy cập dữ liệu liên tục trong quá trình đào tạo dữ liệu mà bạn không cần lo lắng về connection string hoặc data path 
- Bạn có thể version metadata của data asset

Có 3 kiểu chính của data assets mà bạn có thể sử dụng: ```URI File```, ```URI Folder```, ```ML Table```
- URI file: điểm đến của file được chỉ định
- URI folder: điểm đến một folder
- ML Table: Điểm đến một folder hơacj file bao gồm một khung để đọc như dữ liệu dạng bảng

---
# Khởi tạo một URI file data asset

Những paths được hỗ trợ khi sử dụng để tạo một URI file data asset:
- Local: ```./<path>```
- Azure Blob Storage: ```wasbs://<account_name>.blob.core.windows.net/<container_name>/<folder>/<file>```
- Azure Data Lake Storage (Gen 2): ```abfss://<file_system>@<account_name>.dfs.core.windows.net/<folder>/<file>```
- Datastore: ```azureml://datastores/<datastore_name>/paths/<folder>/<file>```

In [7]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_path = "../data/customers-100.csv"

my_data = Data(
    path=my_path,
    type=AssetTypes.URI_FILE,
    description="Data from a CSV file",
    name="local_data",
    version="1.1.1",
)

ml_client.data.create_or_update(my_data)

Data({'path': 'azureml://subscriptions/da6080d2-3021-495c-8ecc-44feb23b1080/resourcegroups/Machine_Learning_dev01/workspaces/Machine_Experiment/datastores/workspaceblobstore/paths/LocalUpload/5403943f29d8a3f7dc9d03d02305860a/customers-100.csv', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'local_data', 'description': 'Data from a CSV file', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/da6080d2-3021-495c-8ecc-44feb23b1080/resourceGroups/Machine_Learning_dev01/providers/Microsoft.MachineLearningServices/workspaces/Machine_Experiment/data/local_data/versions/1.1.1', 'Resource__source_path': '', 'base_path': 'g:\\AzureML_CouresVN\\03', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x000002A25244ED90>, 'serialize': <msrest.serialization.Serializer object at 0x000002A25A24F150>, 'version

---
# Khởi tạo URI Folder data asset

Để khởi tạo URI Folder data asset bạn cần phải có địa chỉ mà folder được chỉ định đang nằm ở đó. Địa chỉ đó có thể là local hoặc cloud

In [16]:
datastore_path = "../data"

my_data = Data(
    path=datastore_path,
    type=AssetTypes.URI_FOLDER,
    description="Data asset pointing to data-asset-path folder in datastore",
    name="diabetes-datastore-path",
)

ml_client.data.create_or_update(my_data)

Data({'path': 'azureml://subscriptions/da6080d2-3021-495c-8ecc-44feb23b1080/resourcegroups/Machine_Learning_dev01/workspaces/Machine_Experiment/datastores/workspaceblobstore/paths/LocalUpload/de51b946ae51d6489d32b8c402371912/data/', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_folder', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'diabetes-datastore-path', 'description': 'Data asset pointing to data-asset-path folder in datastore', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/da6080d2-3021-495c-8ecc-44feb23b1080/resourceGroups/Machine_Learning_dev01/providers/Microsoft.MachineLearningServices/workspaces/Machine_Experiment/data/diabetes-datastore-path/versions/1', 'Resource__source_path': '', 'base_path': 'g:\\AzureML_CouresVN\\03', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x000002A2627CB0D0>, 'serialize': <msrest.serialization.

---
# Khởi tạo ML Table data asset

> **Chú ý**: Nếu chưa cài đặt ml table hãy cài đặt bằng dòng lệnh ```pip``` phía dưới

In [8]:
# !pip install -U mltable azureml-dataprep[pandas]

In [9]:
import mltable
from mltable import MLTableHeaders, MLTableFileEncoding, DataType

# create paths to the data files
paths = [{"file": "wasbs://data@azuremlexampledata.blob.core.windows.net/titanic.csv"}]

# create an MLTable from the data files
tbl = mltable.from_delimited_files(
    paths=paths,
    delimiter=",",
    header=MLTableHeaders.all_files_same_headers,
    infer_column_types=True,
    include_path_column=False,
    encoding=MLTableFileEncoding.utf8,
)

# filter out rows undefined ages
tbl = tbl.filter("col('Age') > 0")

# drop PassengerId
tbl = tbl.drop_columns(["PassengerId"])

# ensure survived column is treated as boolean
data_types = {
    "Survived": DataType.to_bool(
        true_values=["True", "true", "1"], false_values=["False", "false", "0"]
    )
}
tbl = tbl.convert_column_types(data_types)

# show the first 5 records
tbl.show(5)

# You can also load into pandas...
# df = tbl.to_pandas_dataframe()
# df.head(5)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,False,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,True,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,True,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,True,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,False,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


In [11]:
# save the data loading steps in an MLTable file
# NOTE: the tbl object was defined in the previous snippet.
tbl.save("../data/titanic")

paths:
- file: wasbs://data@azuremlexampledata.blob.core.windows.net/titanic.csv
transformations:
- read_delimited:
    delimiter: ','
    empty_as_string: false
    encoding: utf8
    header: all_files_same_headers
    include_path_column: false
    infer_column_types: true
    partition_size: 20971520
    path_column: Path
    support_multi_line: false
- filter: col('Age') > 0
- drop_columns:
  - PassengerId
- convert_column_types:
  - column_type:
      boolean:
        false_values:
        - '0'
        - 'False'
        - 'false'
        mismatch_as: error
        true_values:
        - '1'
        - 'True'
        - 'true'
    columns: Survived
type: mltable

In [12]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

local_path = "../data/titanic"

my_data = Data(
    path=local_path,
    type=AssetTypes.MLTABLE,
    description="MLTable pointing to diabetes.csv in data folder",
    name="diabetes-table",
)
ml_client.data.create_or_update(my_data)

Uploading titanic (0.0 MBs): 100%|##########| 704/704 [00:00<00:00, 701208.74it/s]




Data({'path': 'azureml://subscriptions/da6080d2-3021-495c-8ecc-44feb23b1080/resourcegroups/Machine_Learning_dev01/workspaces/Machine_Experiment/datastores/workspaceblobstore/paths/LocalUpload/5a45a2fb5468da6202be6272df8e7456/titanic/', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': ['wasbs://data@azuremlexampledata.blob.core.windows.net/titanic.csv'], 'type': 'mltable', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'diabetes-table', 'description': 'MLTable pointing to diabetes.csv in data folder', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/da6080d2-3021-495c-8ecc-44feb23b1080/resourceGroups/Machine_Learning_dev01/providers/Microsoft.MachineLearningServices/workspaces/Machine_Experiment/data/diabetes-table/versions/1', 'Resource__source_path': '', 'base_path': 'g:\\AzureML_CouresVN\\03', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x000002A25C4B2910>,

---

# Đọc data trong data assets đã tạo 

In [17]:
import mltable

registered_data_asset = ml_client.data.get(name="diabetes-table", version=1)
tbl = mltable.load(f"azureml:/{registered_data_asset.id}")
df = tbl.to_pandas_dataframe()
df.head(5)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,False,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,True,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,True,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,True,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,False,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


# Test thử trong 1 job

In [ ]:
import os

# create a folder for the script files
script_folder = "src"
os.makedirs(script_folder, exist_ok=True)
print(script_folder, "folder created")

In [ ]:
%%writefile $script_folder/move-data.py
# import libraries
import argparse
import pandas as pd
import numpy as np
from pathlib import Path

def main(args):
    # read data
    df = get_data(args.input_data)

    output_df = df.to_csv((Path(args.output_datastore) / "diabetes.csv"), index = False)

# function that reads the data
def get_data(path):
    df = pd.read_csv(path)

    # Count the rows and print the result
    row_count = (len(df))
    print('Analyzing {} rows of data'.format(row_count))
    
    return df

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--input_data", dest='input_data',
                        type=str)
    parser.add_argument("--output_datastore", dest='output_datastore',
                        type=str)

    # parse args
    args = parser.parse_args()

    # return args
    return args

# run script
if __name__ == "__main__":
    # add space in logs
    print("\n\n")
    print("*" * 60)

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")

In [ ]:
from azure.ai.ml import Input, Output
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import command

# configure input and output
my_job_inputs = {
    "local_data": Input(type=AssetTypes.URI_FILE, path="azureml:diabetes-local:1")
}

my_job_outputs = {
    "datastore_data": Output(type=AssetTypes.URI_FOLDER, path="azureml://datastores/blob_training_data/paths/datastore-path")
}

# configure job
job = command(
    code="./src",
    command="python move-data.py --input_data {{input.local_data}} --output_datastore {{outputs.datastore_data}}",
    inputs=my_job_inputs,
    outputs=my_job_outputs,
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="move-diabetes-data",
    experiment_name="move-diabetes-data",
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)